<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_Ensemble_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retriever

**Ensemble = (sparse + Dense) + reorder**

sparse는 키워드를 중심으로 정리하고 검색한다. dense는 문장을 중심으로 정리하고 검색한다. 즉 하나의 키워드라면 sparse가 더 찾을 수 있고, 그걸 어떤 키워드를 문맥적으로 파악하는 것은 dense이다.
그래서 sparse는 이음동의어를 찾을 수 없지만, dense는 이음동의어를 처리할 수 있게 된다.
Ensemble Retriever : 위의 2가지 방식을 써서... 더 정확한 답을 얻을 수 있도록 한다.  


Long context Reorder : 보통 일을 할 때, 도입부에 신경쓰고 마무리에 신경 좀 쓰면 최소노력으로 최대효과를 볼 수 있는데 AI가 똑똑하게도, 주어진 문서 중에 첫 3개정도와 마지막 몇개를 가장 집중적으로 참고한다고 한다(U자 모양) 그래서.. 중요한 문서일 수록 이 앞뒤에 배치해줄 수 있도록 하는 기술.

In [ ]:
# colab 환경설정
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

from google.colab import drive as gd
drive.mount('/content/drive')

In [ ]:
pip install --quiet icecream > /dev/null

In [ ]:
import pprint
from icecream import ic

def pp(object):
  ppr = pprint.PrettyPrinter(
      # indent=40,
      width=80
      )
  return ppr.pprint(object)

In [ ]:
from IPython.display import clear_output

# Clear all output cells
clear_output()

# Ensenble

In [ ]:
pip install -q langchain langchain-openai pypdf sentence-transformers chromadb  faiss-cpu -U rank_bm25 > /dev/null

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# ___ setting ___
LLM_MODEL = "gpt-3.5-turbo"
MAX = 265
TEMP = 1.5
# _______________

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

openai_compbot = OpenAI(
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)
openai_chatbot = ChatOpenAI(
    model_name = LLM_MODEL,
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)

ko_embed = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sbert-nli",
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
    )

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

SOURCE_folder = "/content/drive/MyDrive/data/brand"
SOURCE = "/content/source/1-1그로스 해킹, 마케팅과 어떻게 다른가요_ - PUBLY.pdf"

loader = DirectoryLoader(
    SOURCE_folder,
    glob='**/*.pdf',
    show_progress=True,
    loader_cls=PyPDFLoader,
    )

pages = loader.load_and_split()

# 이건 여러번의 로더 호출을 통해, 각 문서가 각각의 로더객체에 담겼을 경우,
# 이 모두를 합해서 하나의 doc 객체로 만들어서 downstream에서 이용하기 위해 합쳐주는 기능.
# 따라서... 우리는 디렉토리 로더로 어차피 하나의 객체에 모든 문서를 '페이지별'로 담았다.
# 이 pypdf는 내부적으로 리컬시브~를 쓴다고 한다.
# docs = []
# for page in pages:
#     docs.append(page.page_content)

ic(len(pages))

In [ ]:
# 이걸 문서별로 덩어리로 만들고 싶다. 즉 불러온 문서가 3개라면, 객체속의 요소도 3개만 존재하도록.
# keys = []
# for page in pages:
#     source = page.metadata['source']
#     if source not in keys:
#         keys.append(page.metadata['source'])
# keys

# 그러나 지금은 일단 포기

lens = []
for page in pages:
    lens.append(len(page.page_content))

print(sorted(lens, reverse = True), end=' ')
print('\n','*'*100, '\n')
print(min(lens), max(lens))

docs = pages

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 적정한 길이로 잘라주고
rc_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=30,
    length_function=len,
)

# chunks = rc_splitter.create_documents(docs) # 옵션을 줄 수 있다. []을 받는다.
chunks = rc_splitter.split_documents(docs) # 옵션을 줄 수 있다. []을 받는다.

len(chunks)

lens = []
for chunk in chunks:
    lens.append(len(chunk.page_content))

print(sorted(lens, reverse = True), end=' ')
print('\n','*'*100, '\n')
print(min(lens), max(lens))

In [ ]:
# for Sparse retriever
bm25_ret = BM25Retriever.from_documents(chunks)
bm25_ret.k=2 # 여기는 선언방식이 진짜 특이하다.

#for Dense retriever
embedding = ko_embed
faiss_db = FAISS.from_documents(chunks, embedding)
faiss_ret = faiss_db.as_retriever(search_kwargs={'k':2})

#init ensemble retriever
ensemble_ret = EnsembleRetriever(
    retrievers = [bm25_ret, faiss_ret], weight=[0.5,0.5]
)

In [ ]:
# 앙상블은 양쪽 리트리버에서 결과를 가져와서
# 순위별로 재정렬도 해준다.

query = '브랜드 네이밍과 인스타그램'

doc = ensemble_ret.invoke(query)

for i in docs:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
docs_faiss = faiss_ret.invoke(query)

for i in docs_faiss:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
docs_bm25 = bm25_ret.invoke(query)

for i in docs_bm25:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm = openai_compbot,
    chain_type = 'stuff',
    retriever = faiss_ret,
    return_source_documents = True
)

result_faiss = qa(query)
print(result_faiss['result'])

for i in result_faiss['source_documents']:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm = openai_compbot,
    chain_type = 'stuff',
    retriever = bm25_ret,
    return_source_documents = True
)

result_bm25 = qa(query)
print(result_bm25['result'])

for i in result_bm25['source_documents']:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm = openai_compbot,
    chain_type = 'stuff',
    retriever = ensemble_ret,
    return_source_documents = True
)

result = qa(query)
print(result['result'])

for i in result['source_documents']:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

3개의 리트리버를 비교해보면, sparse에서 가장 키워드를 잘 찾아오고(당연한 이야기지만) 문맥적으로는 dense에서 찾는 것을 알 수 있고, 그 순서대로 ensemble에 반영되어 문장이 만들어진다는 것을 확인할 수 있다.